In [2]:
import arcgis
from arcgis.gis import GIS
import os, re, csv

#Define variables
   
portalURL = r'https://gissp.bracell.com/portal/'
username = "Qualidade_Florestal"
password = "Bracell@21"

    #Form Id:
survey_item_id = "2897ffb3737d4ba7bf556ec3ab3e1770"

save_path = r"F:/Qualidade_Florestal/03- ADMINISTRATIVO/2023/06- COLABORADORES/Gabriel/1 - Processamento de dados"

keep_org_item = False
store_csv_w_attachments = False


# Conecta ao PORTAL GIS e identifica o Formulário:

# In[ ]:


# Connect
gis = GIS(portalURL, username, password)
survey_by_id = gis.content.get(survey_item_id)
print(survey_by_id.type)
survey_by_id


# Download do serviço:

# In[ ]:


# Download xslx
rel_fs = survey_by_id.related_items('Survey2Service','forward')[0]
rel_fs


# In[ ]:


item_excel = rel_fs.export(title=survey_by_id.title, export_format='Excel')
item_excel.download(save_path=save_path)
if not bool(keep_org_item):
    item_excel.delete(force=True)


# Criação da pasta de fotos e arquivo catálogo em csv. no diretório definido. Caso a pasta já exista, fará uma substituição:

# In[ ]:


layers = rel_fs.layers + rel_fs.tables
for i in layers:
    if i.properties.hasAttachments == True:
        print("Item: ", i, "TEM FOTO")
        feature_layer_folder = os.path.join(save_path, '{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)))
        if not os.path.exists(feature_layer_folder):
            os.mkdir(feature_layer_folder)         
        if bool(store_csv_w_attachments):
            path = os.path.join(feature_layer_folder, "{}_attachments.csv".format(i.properties.name))
        elif not bool(store_csv_w_attachments):
            path = os.path.join(save_path, "{}_attachments.csv".format(i.properties.name))
        csv_fields = ['Parent objectId', 'Attachment path']
        with open(path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(csv_fields)
            
            feature_object_ids = i.query(where="1=1", return_ids_only=True, order_by_fields='objectid ASC')
            for j in range(len(feature_object_ids['objectIds'])):
                current_oid = feature_object_ids['objectIds'][j]
                current_oid_attachments = i.attachments.get_list(oid=current_oid)
            
                if len(current_oid_attachments) > 0:
                    for k in range(len(current_oid_attachments)):
                        attachment_id = current_oid_attachments[k]['id']
                        current_attachment_path = i.attachments.download(oid=current_oid, attachment_id=attachment_id, save_path=feature_layer_folder)
                        csvwriter.writerow([current_oid, os.path.join('{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)), os.path.split(current_attachment_path[0])[1])])
    else:
        print("Item: ", i, "NÃO TEM FOTO")
        
    
print('Concluído!')
            


# Ao final, as mensagens indicarão as respostas dos formulários que não tem foto e quando tiver concluído.

Form
Item:  <FeatureLayer url:"https://gissp.bracell.com/server/rest/services/Hosted/service_aca00316cff54d968313b0963240bc60/FeatureServer/0"> NÃO TEM FOTO
Item:  <Table url:"https://gissp.bracell.com/server/rest/services/Hosted/service_aca00316cff54d968313b0963240bc60/FeatureServer/1"> NÃO TEM FOTO
Item:  <Table url:"https://gissp.bracell.com/server/rest/services/Hosted/service_aca00316cff54d968313b0963240bc60/FeatureServer/2"> TEM FOTO
Item:  <Table url:"https://gissp.bracell.com/server/rest/services/Hosted/service_aca00316cff54d968313b0963240bc60/FeatureServer/3"> TEM FOTO
Concluído!
